In [ ]:
### 1. Mount Google Drive ###

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
### 2. Prepare data ###

!scp '/content/gdrive/My Drive/TesseractVSEasyOCRVSAWSTextract/data.zip' '/content/data.zip'

!unzip '/content/data.zip' -d '/content/'

In [1]:
### 3. Install dependencies ###

!apt install tesseract-ocr
!apt install libtesseract-dev

!pip install pytesseract
!pip install Pillow
!pip install easyocr
!pip install boto3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.3 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
# Text detection

import pytesseract
from PIL import Image

image_path = '/content/data/it_was_all_a_dream.jpg'

text = pytesseract.image_to_string(Image.open(image_path), lang='eng')

print(text)

In [ ]:
from easyocr import Reader

reader = Reader(['en'])
result = reader.readtext(Image.open(image_path)
for results in result:
  text = text + results[1] + ''

text = text[:-1]

print(text)

In [ ]:
from http import client
import boto3

access_key = ''
secret_access_key = ''

textract_client = boto3.client('textract',
                               aws_access_key_id=access_key,
                               aws_secret_access_key=secret_access_key,
                               region_name='us-east-1')

with open(image_path, 'rb') as in:
  response = textract_client.detect_document_text(Document={'Bytes': in.read()})

text = ''

for block in response['Blocks']:
  if block['BlockType'] == 'LINE':
    text = text + block['Text'] + ''

text = text[:-1]

print(text)


In [2]:
# Compare Performance

def jaccard_similarity(sentence1, sentence2):
    # Split the sentences into sets of words
    set1 = set(sentence1.split())
    set2 = set(sentence2.split())

    # Calculate the intersection and union of the two sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Calculate Jaccard similarity
    if not union:  # Handle the case where both sets are empty
        return 0.0

    jaccard_index = len(intersection) / len(union)
    return jaccard_index

# Example usage
sentence1 = "I love programming"
sentence2 = "I love coding"
similarity = jaccard_similarity(sentence1, sentence2)
print(f"Jaccard Similarity: {similarity}")

Jaccard Similarity: 0.5
